### Import required libraries

In [9]:
import json
import networkx as nx
import openai
from groq import Groq
from neo4j import GraphDatabase
import json
import json
import networkx as nx
import time

### Connection Neo4j 

In [10]:
# Neo4j connection details
uri = "bolt://localhost:7687"  
user = "neo4j"                 
password = "123456789"          

# Initialize the Neo4j driver
driver = GraphDatabase.driver(uri, auth=(user, password))

# Function to run a query and return the results
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

### Get all dataa

In [22]:
# Get all nodes
def get_all_nodes(tx):
    query = """
    MATCH (n)
    RETURN labels(n) AS labels, n
    """
    result = tx.run(query)
    nodes = []
    for record in result:
        labels = record["labels"]
        node = record["n"]
        nodes.append({
            "labels": labels,
            "properties": dict(node)
        })
    return nodes

# Perform query
with driver.session() as session:
    all_nodes = session.read_transaction(get_all_nodes)

# Close connection driver
driver.close()

# Result
for node in all_nodes:
    print(f"Labels: {node['labels']}, Properties: {node['properties']}")

Labels: ['Group', 'HighValue'], Properties: {'highvalue': True, 'neo4jImportId': '0', 'domain': 'TestCompany.Local', 'name': 'DOMAIN ADMINS@TestCompany.Local', 'objectid': 'S-1-5-21-883232822-274137685-4173207997-512'}
Labels: ['Group'], Properties: {'neo4jImportId': '1', 'domain': 'TestCompany.Local', 'name': 'DOMAIN COMPUTERS@TestCompany.Local', 'objectid': 'S-1-5-21-883232822-274137685-4173207997-515'}
Labels: ['Group'], Properties: {'neo4jImportId': '2', 'domain': 'TestCompany.Local', 'name': 'DOMAIN USERS@TestCompany.Local', 'objectid': 'S-1-5-21-883232822-274137685-4173207997-513'}
Labels: ['Group', 'HighValue'], Properties: {'highvalue': True, 'neo4jImportId': '3', 'domain': 'TestCompany.Local', 'name': 'DOMAIN CONTROLLERS@TestCompany.Local', 'objectid': 'S-1-5-21-883232822-274137685-4173207997-516'}
Labels: ['Group', 'HighValue'], Properties: {'highvalue': True, 'neo4jImportId': '4', 'domain': 'TestCompany.Local', 'name': 'ENTERPRISE DOMAIN CONTROLLERS@TestCompany.Local', 'obje

/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_87775/2248797696.py:19: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_87775/2248797696.py:20: DeprecationWarning: read_transaction has been renamed to execute_read
  all_nodes = session.read_transaction(get_all_nodes)


### Graph Encoding

In [14]:
def encode_data(data):
    encoded_str = ""
    for idx, node in enumerate(data):
        labels_str = ', '.join(node['labels'])
        properties_str = ', '.join([f"{key}: {value}" for key, value in node['properties'].items()])
        encoded_str += f"Node {idx} has labels [{labels_str}] and ({properties_str}), "
    return encoded_str[:-2]  # Loại bỏ dấu phẩy cuối cùng

# Encode data and print results
encoded_all_data = encode_data(all_nodes)
print(encoded_all_data)

Node 0 has labels [Group, HighValue] and (highvalue: True, neo4jImportId: 0, domain: TestCompany.Local, name: DOMAIN ADMINS@TestCompany.Local, objectid: S-1-5-21-883232822-274137685-4173207997-512), Node 1 has labels [Group] and (neo4jImportId: 1, domain: TestCompany.Local, name: DOMAIN COMPUTERS@TestCompany.Local, objectid: S-1-5-21-883232822-274137685-4173207997-515), Node 2 has labels [Group] and (neo4jImportId: 2, domain: TestCompany.Local, name: DOMAIN USERS@TestCompany.Local, objectid: S-1-5-21-883232822-274137685-4173207997-513), Node 3 has labels [Group, HighValue] and (highvalue: True, neo4jImportId: 3, domain: TestCompany.Local, name: DOMAIN CONTROLLERS@TestCompany.Local, objectid: S-1-5-21-883232822-274137685-4173207997-516), Node 4 has labels [Group, HighValue] and (highvalue: True, neo4jImportId: 4, domain: TestCompany.Local, name: ENTERPRISE DOMAIN CONTROLLERS@TestCompany.Local, objectid: TestCompany.Local-S-1-5-9), Node 5 has labels [Group] and (neo4jImportId: 5, domain:

In [15]:
len(encoded_all_data)

214020

#### Get relationships and properties¶

In [16]:
def get_nodes_and_relationships(tx):
    query = (
        "MATCH (a)-[r]->(b) "
        "RETURN DISTINCT labels(a) AS StartLabels, type(r) AS RelationshipType, labels(b) AS EndLabels, properties(r) AS RelationshipProperties "
        "ORDER BY StartLabels, RelationshipType, EndLabels"
    )
    result = tx.run(query)
    nodes_and_relationships = []
    for record in result:
        nodes_and_relationships.append({
            "StartLabels": record["StartLabels"],
            "RelationshipType": record["RelationshipType"],
            "EndLabels": record["EndLabels"],
            "RelationshipProperties": dict(record["RelationshipProperties"])  # Convert properties to dictionary
        })
    return nodes_and_relationships


with driver.session() as session:
    nodes_and_relationships = session.read_transaction(get_nodes_and_relationships)

# Close connection to driver
driver.close()

/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_87775/3342820478.py:19: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_87775/3342820478.py:20: DeprecationWarning: read_transaction has been renamed to execute_read
  nodes_and_relationships = session.read_transaction(get_nodes_and_relationships)


In [17]:
nodes_and_relationships

[{'StartLabels': ['Computer'],
  'RelationshipType': 'HAS_SESSION',
  'EndLabels': ['User'],
  'RelationshipProperties': {}},
 {'StartLabels': ['Computer'],
  'RelationshipType': 'MEMBER_OF',
  'EndLabels': ['Group'],
  'RelationshipProperties': {}},
 {'StartLabels': ['Computer'],
  'RelationshipType': 'MEMBER_OF',
  'EndLabels': ['Group', 'HighValue'],
  'RelationshipProperties': {}},
 {'StartLabels': ['Domain', 'HighValue'],
  'RelationshipType': 'CONTAINS',
  'EndLabels': ['OU'],
  'RelationshipProperties': {}},
 {'StartLabels': ['Domain', 'HighValue'],
  'RelationshipType': 'CONTAINS',
  'EndLabels': ['OU'],
  'RelationshipProperties': {'isacl': False}},
 {'StartLabels': ['Domain', 'HighValue'],
  'RelationshipType': 'GP_LINK',
  'EndLabels': ['OU'],
  'RelationshipProperties': {}},
 {'StartLabels': ['GPO'],
  'RelationshipType': 'GP_LINK',
  'EndLabels': ['Domain', 'HighValue'],
  'RelationshipProperties': {'enforced': False, 'isacl': False}},
 {'StartLabels': ['GPO'],
  'Relation

### Edges Encoding

In [19]:
def encode_relationship(relationship):
    start_labels = ', '.join(relationship['StartLabels'])
    end_labels = ', '.join(relationship['EndLabels'])
    relationship_type = relationship['RelationshipType']
    properties = relationship['RelationshipProperties']

    properties_str = ', '.join([f"{key}: {value}" for key, value in properties.items()]) if properties else "No properties"

    return f"Node [{start_labels}] connect to node  [{end_labels}] with Relationship Type {relationship_type} and Properties {{{properties_str}}}"

encoded_relationship =  ""
# Print
for relationship in nodes_and_relationships:
    encoded_relationship += encode_relationship(relationship) + ". "

In [20]:
encoded_relationship

'Node [Computer] connect to node  [User] with Relationship Type HAS_SESSION and Properties {No properties}. Node [Computer] connect to node  [Group] with Relationship Type MEMBER_OF and Properties {No properties}. Node [Computer] connect to node  [Group, HighValue] with Relationship Type MEMBER_OF and Properties {No properties}. Node [Domain, HighValue] connect to node  [OU] with Relationship Type CONTAINS and Properties {No properties}. Node [Domain, HighValue] connect to node  [OU] with Relationship Type CONTAINS and Properties {isacl: False}. Node [Domain, HighValue] connect to node  [OU] with Relationship Type GP_LINK and Properties {No properties}. Node [GPO] connect to node  [Domain, HighValue] with Relationship Type GP_LINK and Properties {enforced: False, isacl: False}. Node [GPO] connect to node  [OU] with Relationship Type GP_LINK and Properties {enforced: False, isacl: False}. Node [GPO] connect to node  [OU] with Relationship Type GP_LINK and Properties {No properties}. Nod

### data Preparing

In [44]:
encoded_data = encoded_all_data[0:80000] +  "\nIn this graph: \n" + encoded_relationship 

In [31]:
encoded_data

'Node 0 has labels [Group, HighValue] and (highvalue: True, neo4jImportId: 0, domain: TestCompany.Local, name: DOMAIN ADMINS@TestCompany.Local, objectid: S-1-5-21-883232822-274137685-4173207997-512), Node 1 has labels [Group] and (neo4jImportId: 1, domain: TestCompany.Local, name: DOMAIN COMPUTERS@TestCompany.Local, objectid: S-1-5-21-883232822-274137685-4173207997-515), Node 2 has labels [Group] and (neo4jImportId: 2, domain: TestCompany.Local, name: DOMAIN USERS@TestCompany.Local, objectid: S-1-5-21-883232822-274137685-4173207997-513), Node 3 has labels [Group, HighValue] and (highvalue: True, neo4jImportId: 3, domain: TestCompany.Local, name: DOMAIN CONTROLLERS@TestCompany.Local, objectid: S-1-5-21-883232822-274137685-4173207997-516), Node 4 has labels [Group, HighValue] and (highvalue: True, neo4jImportId: 4, domain: TestCompany.Local, name: ENTERPRISE DOMAIN CONTROLLERS@TestCompany.Local, objectid: TestCompany.Local-S-1-5-9), Node 5 has labels [Group] and (neo4jImportId: 5, domain

#### Rule Generation

In [25]:
client = Groq(
    api_key="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",
)

In [26]:
def generator(encoded_graph, query, model, size):
    # Function to divide the text into smaller chunks
    def chunk_text(text, max_length):
        words = text.split()
        chunks = []
        current_chunk = []
        current_length = 0

        for word in words:
            if current_length + len(word) + 1 > max_length:
                chunks.append(' '.join(current_chunk))
                current_chunk = []
                current_length = 0
            current_chunk.append(word)
            current_length += len(word) + 1

        if current_chunk:
            chunks.append(' '.join(current_chunk))

        return chunks

    # Function to process each chunk
    def process_chunk(chunk):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": chunk,
                }
            ],
            model= model,
        )

        return chat_completion.choices[0].message.content

    # Split encoded_graph into chunks
    max_chunk_length = size  # Adjust this length based on your needs and model limits
    graph_chunks = chunk_text(encoded_graph, max_chunk_length)

    # Process each chunk of encoded_graph
    processed_graph_parts = [process_chunk(chunk) for chunk in graph_chunks]

    # Combine processed parts back into a single string
    processed_graph = ' '.join(processed_graph_parts)

    # Append query to the processed graph and process the final combined context
    context = processed_graph + "\n" + query
    final_result = process_chunk(context)

    return final_result

### Zero-shot Prompt

In [27]:
zero_prompt = """
Based on the following graph properties, generate detailed consistency rules (graph functional dependency and graph entity dependency).
Consider the structure, node information and relationships in the graph, and provide a set of rules that can be applied to maintain consistent 
and accurate data.

For each consistency rule you identify, provide a clear description of the rule and the corresponding Cypher query
"""

### LLAMA

In [32]:
rules_llama = generator(encoded_data, zero_prompt, "llama3-70b-8192", 8000)

In [33]:
print(rules_llama)

What a delightful dump of Neo4j nodes!

After analyzing the nodes and their properties, I've identified some potential consistency rules to maintain accurate and consistent data. Here are the rules I came up with:

**1. Unique Node IDs**
 Rule: Each node should have a unique `neo4jImportId`.
Cypher Query: `CREATE CONSTRAINT ON (n) ASSERT n.neo4jImportId IS UNIQUE`

**2. Domain Consistency**
 Rule: All nodes with a `domain` property should have a valid domain name.
Cypher Query: `MATCH (n) WHERE n.domain IS NOT NULL AND NOT n.domain =~ '^[a-zA-Z0-9.-]+$' RETURN n`

**3. Owned Property Consistency**
 Rule: The `owned` property should only be `True` or `False`.
Cypher Query: `MATCH (n) WHERE n.owned IS NOT NULL AND NOT (n.owned = True OR n.owned = False) RETURN n`

**4. Enabled Property Consistency**
 Rule: The `enabled` property should only be `True` or `False`.
Cypher Query: `MATCH (n) WHERE n.enabled IS NOT NULL AND NOT (n.enabled = True OR n.enabled = False) RETURN n`

**5. User Node 

### Evaluation: 

#### 1. Unique Node IDs
**Rule: Each node should have a unique `neo4jImportId`**

- **Calculate support (953)**
  
        MATCH (n)
        WITH n.neo4jImportId AS id, COUNT(n) AS countNodes
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN COUNT(id) AS support

- **Calculate coverage (1.0)**
  
        MATCH (n)
        WITH COUNT(n) AS totalNodes
        MATCH (n)
        WITH n.neo4jImportId AS id, COUNT(n) AS countNodes, totalNodes
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN toFloat(COUNT(id)) / totalNodes AS coverage

- **Confidence: 1.0**

        MATCH (n)
        WHERE n.neo4jImportId IS NOT NULL
        WITH COUNT(n) AS totalNodesWithId
        MATCH (n)
        WITH n.neo4jImportId AS id, COUNT(n) AS countNodes, totalNodesWithId
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN toFloat(COUNT(id)) / totalNodesWithId AS confidence


#### 2. Domain Consistency
**Rule: All nodes with a `domain` property should have a valid domain name**

- **Calculate support (953)**
  
        MATCH (n)
        WHERE EXISTS(n.domain) AND toString(n.domain) = n.domain
        RETURN COUNT(n) AS support

- **Calculate coverage (1.0)**
  
        MATCH (n)
        WHERE EXISTS(n.domain)
        WITH COUNT(n) AS totalNodesWithDomain
        MATCH (n)
        WHERE EXISTS(n.domain) AND toString(n.domain) = n.domain
        RETURN toFloat(COUNT(n)) / totalNodesWithDomain AS coverage

- **Calculate confidence (1.0)**
  
        MATCH (n)
        WHERE EXISTS(n.domain)
        WITH COUNT(n) AS totalNodesWithDomain
        MATCH (n)
        WHERE EXISTS(n.domain) AND toString(n.domain) = n.domain
        RETURN toFloat(COUNT(n)) / totalNodesWithDomain AS confidence


#### 3. Owned Property Consistency
**Rule: The `owned` property should only be `True` or `False`**

- **Calculate support (603)**
  
        MATCH (n)
        WHERE EXISTS(n.owned) AND (n.owned = true OR n.owned = false)
        RETURN COUNT(n) AS support


- **Calculate coverage (0.63)**
  
        MATCH (n)
        WITH COUNT(n) AS totalNodes
        MATCH (n)
        WHERE EXISTS(n.owned) AND (n.owned = true OR n.owned = false)
        RETURN toFloat(COUNT(n)) / totalNodes AS coverage


	
- **Calculate confidence (1.0)**
  
        MATCH (n)
        WHERE EXISTS(n.owned)
        WITH COUNT(n) AS totalNodesWithOwned
        MATCH (n)
        WHERE EXISTS(n.owned) AND (n.owned = true OR n.owned = false)
        RETURN toFloat(COUNT(n)) / totalNodesWithOwned AS confidence


#### 4. Enabled Property Consistency
**Rule: The `enabled` property should only be `True` or `False`**

- **Calculate support (600)**
  
        MATCH (n)
        WHERE EXISTS(n.enabled) AND (n.enabled = true OR n.enabled = false)
        RETURN COUNT(n) AS support

- **Calculate coverage (0.63)**
  
        MATCH (n)
        WITH COUNT(n) AS totalNodes
        MATCH (n)
        WHERE EXISTS(n.enabled) AND (n.enabled = true OR n.enabled = false)
        RETURN toFloat(COUNT(n)) / totalNodes AS coverage


	
- **Calculate confidence (1.0)**
  
        MATCH (n)
        WHERE EXISTS(n.enabled)
        WITH COUNT(n) AS totalNodesWithOwned
        MATCH (n)
        WHERE EXISTS(n.enabled) AND (n.enabled = true OR n.enabled = false)
        RETURN toFloat(COUNT(n)) / totalNodesWithOwned AS confidence

#### 5. User Node Consistency
**Rule: User nodes should have a valid `displayname` and `username`**

- **Calculate support (300)**
  
        MATCH (n:User)
        WHERE EXISTS(n.name)
        RETURN COUNT(n) AS support

- **Calculate coverage (1.0)**
  
        MATCH (n:User)
        WITH COUNT(n) AS totalNodes
        MATCH (n:User)
        WHERE EXISTS(n.name)
        RETURN toFloat(COUNT(n)) / totalNodes AS coverage

- **Calculate confidence (1.0)**
  
        MATCH (n:User)
        WITH COUNT(n) AS totalNodes
        MATCH (n:User)
        WHERE EXISTS(n.name)
        RETURN toFloat(COUNT(n)) / totalNodes AS confidence


#### 6. Computer Node Consistency
**Rule: Computer nodes should have a valid `operatingsystem` and `domain`**

- **Calculate support (301)**
  
        MATCH (c:Computer)
        WHERE EXISTS(c.domain)
        RETURN COUNT(c) AS support

- **Calculate coverage (1.0)**
  
        MATCH (c:Computer)
        WITH COUNT(c) AS totalNodes
        MATCH (c:Computer)
        WHERE EXISTS(c.domain)
        RETURN toFloat(COUNT(c)) / totalNodes AS coverage

- **Calculate confidence (1.0)**
  
        MATCH (c:Computer)
        WITH COUNT(c) AS totalNodes
        MATCH (c:Computer)
        WHERE EXISTS(c.domain)
        RETURN toFloat(COUNT(c)) / totalNodes AS confidence



#### 7. Group Node Consistency
**Rule: Group nodes should have a valid `displayname` and `domain`**

- **Calculate support (308)**
  
        MATCH (g:Group)
        WHERE EXISTS(g.domain)
        RETURN COUNT(g) AS support

- **Calculate coverage (1.0)**
  
        MATCH (g:Group)
        WITH COUNT(g) AS totalNodes
        MATCH (g:Group)
        WHERE EXISTS(g.domain)
        RETURN toFloat(COUNT(g)) / totalNodes AS coverage

- **Calculate confidence (1.0)**
  
        MATCH (g:Group)
        WITH COUNT(n) AS totalNodes
        MATCH (g:Group)
        WHERE EXISTS(g.domain)
        RETURN toFloat(COUNT(g)) / totalNodes AS confidence



#### 8. OU Node Consistency
**Rule: OU nodes should have a valid `displayname` and `domain`**

- **Support: 21**
  
        MATCH (ou:OU)
        WHERE  EXISTS(ou.domain)
        RETURN COUNT(ou) AS valid_ou_nodes

- **Coverage: 1.0**

        // Calculate confidence as the ratio of valid OU nodes to total OU nodes
        MATCH (ou:OU)
        WHERE EXISTS(ou.domain)
        WITH COUNT(ou) AS valid_ou_nodes
        
        MATCH (ou:OU)
        WITH valid_ou_nodes, COUNT(ou) AS total_ou_nodes
        
        RETURN toFloat(valid_ou_nodes) / total_ou_nodes AS coverage

- **Confidence: 1.0**

        // Calculate confidence as the ratio of valid OU nodes to total OU nodes
        MATCH (ou:OU)
        WHERE EXISTS(ou.domain)
        WITH COUNT(ou) AS valid_ou_nodes
        
        MATCH (ou:OU)
        WITH valid_ou_nodes, COUNT(ou) AS total_ou_nodes
        
        RETURN toFloat(valid_ou_nodes) / total_ou_nodes AS confident

#### 9. GPO Node Consistency 
**Rule: GPO nodes should have a valid `displayname` and `domain`**

- **Support: 22**
  
        MATCH (g:GPO)
        WHERE  EXISTS(g.domain)
        RETURN COUNT(g) AS valid_ou_nodes

- **Coverage: 1.0**

        // Calculate confidence as the ratio of valid OU nodes to total OU nodes
        MATCH (g:GPO)
        WHERE EXISTS(g.domain)
        WITH COUNT(g) AS valid_g_nodes
        
        MATCH (g:GPO)
        WITH valid_g_nodes, COUNT(g) AS total_g_nodes
        
        RETURN toFloat(valid_g_nodes) / total_g_nodes AS coverage

- **Confidence: 1.0**

        // Calculate confidence as the ratio of valid OU nodes to total OU nodes
        MATCH (g:GPO)
        WHERE EXISTS(g.domain)
        WITH COUNT(g) AS valid_ou_nodes
        
        MATCH (g:GPO)
        WITH valid_g_nodes, COUNT(ou) AS total_g_nodes
        
        RETURN toFloat(valid_g_nodes) / total_g_nodes AS confident

#### 10. Relationship Consistency 
**Rule: Relationships should only connect nodes of compatible types (e.g., User-Computer, Group-User, etc.)**

- Support = 0

### MIXTRAL

In [46]:
rules_mixtral = generator(encoded_data, zero_prompt, "mixtral-8x7b-32768", 8000)

In [50]:
print(rules_mixtral)

Graph Functional Dependency Rules:

1. If a node has the label `Computer`, it must have the properties `operatingsystem`, `name`, `objectid`, and `enabled`.

   Cypher Query:
   ```cypher
   MATCH (c:Computer)
   WHERE NOT (c.operatingsystem IS NOT NULL AND c.name IS NOT NULL AND c.objectid IS NOT NULL AND c.enabled IS NOT NULL)
   RETURN c
   ```

2. If a node has the label `User`, it must have the properties `displayname`, `pwdlastset`, `domain`, `lastlogon`, `name`, `objectid`, and `enabled`.

   Cypher Query:
   ```cypher
   MATCH (u:User)
   WHERE NOT (u.displayname IS NOT NULL AND u.pwdlastset IS NOT NULL AND u.domain IS NOT NULL AND u.lastlogon IS NOT NULL AND u.name IS NOT NULL AND u.objectid IS NOT NULL AND u.enabled IS NOT NULL)
   RETURN u
   ```

3. If a relationship has the type `HAS_SESSION`, it must involve a `Computer` node and a `User` node.

   Cypher Query:
   ```cypher
   MATCH ()-[r:HAS_SESSION]-()
   WHERE NOT (r.source:Computer AND r.target:User)
   RETURN r
   `

### Evaluation

#### Rule 1. 
**If a node has the label `Computer`, it must have the properties `operatingsystem`, `name`, `objectid`, and `enabled`**

- **support: 300**

        MATCH (c:Computer)
        WHERE EXISTS(c.operatingsystem) AND EXISTS(c.name) AND EXISTS(c.objectid) AND EXISTS(c.enabled)
        RETURN COUNT(c) AS support

- **Coverage: 97.2**

        // Calculate coverage
        MATCH (c:Computer)
        WHERE EXISTS(c.operatingsystem) AND EXISTS(c.name) AND EXISTS(c.objectid) AND EXISTS(c.enabled)
        WITH COUNT(c) AS valid_computers
        
        MATCH (c:Computer)
        WITH valid_computers, COUNT(c) AS total_computers
        
        RETURN toFloat(valid_computers) / total_computers AS coverage


- **Confidence: 99.67**

        // Calculate confidence
        MATCH (c:Computer)
        WHERE EXISTS(c.operatingsystem) AND EXISTS(c.name) AND EXISTS(c.objectid) AND EXISTS(c.enabled)
        WITH COUNT(c) AS valid_computers
        
        MATCH (c:Computer)
        WITH valid_computers, COUNT(c) AS total_computers
        
        RETURN toFloat(valid_computers) / total_computers AS confidence


#### Rule 2. 
**If a node has the label `User`, it must have the properties `displayname`, `pwdlastset`, `domain`, `lastlogon`, `name`, `objectid`, and `enabled`**

- **Support: 299**

        // Count nodes with label 'User' that have all required properties
        MATCH (u:User)
        WHERE EXISTS(u.displayname) AND EXISTS(u.pwdlastset) AND EXISTS(u.domain)
          AND EXISTS(u.lastlogon) AND EXISTS(u.name) AND EXISTS(u.objectid)
          AND EXISTS(u.enabled)
        RETURN COUNT(u) AS support

- **Coverage: 99.67**

        // Calculate the number of User nodes with all required properties
        MATCH (u:User)
        WHERE EXISTS(u.displayname) AND EXISTS(u.pwdlastset) AND EXISTS(u.domain)
          AND EXISTS(u.lastlogon) AND EXISTS(u.name) AND EXISTS(u.objectid)
          AND EXISTS(u.enabled)
        WITH COUNT(u) AS valid_users
        
        // Calculate the total number of User nodes
        MATCH (u:User)
        WITH valid_users, COUNT(u) AS total_users
        
        RETURN toFloat(valid_users) / total_users AS coverage


- ***Confidence: 99.76*

        // Calculate the number of User nodes with all required properties
        MATCH (u:User)
        WHERE EXISTS(u.displayname) AND EXISTS(u.pwdlastset) AND EXISTS(u.domain)
          AND EXISTS(u.lastlogon) AND EXISTS(u.name) AND EXISTS(u.objectid)
          AND EXISTS(u.enabled)
        WITH COUNT(u) AS valid_users
        
        // Count the total number of User nodes (i.e., the entire population we're testing)
        MATCH (u:User)
        WITH valid_users, COUNT(u) AS total_users
        
        RETURN toFloat(valid_users) / total_users AS confidence



#### Rule 3. 
**If a relationship has the type `HAS_SESSION`, it must involve a `Computer` node and a `User` node.**

- **Support: 264**

        MATCH (c:Computer)-[r:HAS_SESSION]->(u:User)
        RETURN COUNT(r) AS support

- **Coverage: 1.0**

        // Find relationships of type HAS_SESSION that involve a Computer and a User
        MATCH (c:Computer)-[r:HAS_SESSION]->(u:User)
        WITH COUNT(r) AS valid_relationships
        
        // Count the total number of HAS_SESSION relationships regardless of the nodes
        MATCH ()-[r:HAS_SESSION]->()
        WITH valid_relationships, COUNT(r) AS total_relationships
        
        RETURN toFloat(valid_relationships) / total_relationships AS coverage


- **Confidence: 1.0**
  

#### Rule 4. 
**If a relationship has the type `MEMBER_OF`, it must connect a `User` node to a `Group` node or a `Domain` node (excluding relationships between `Domain` nodes and `OU` nodes).**

- **Support: 1043**

        MATCH (u:User)-[r:MEMBER_OF]->(g:Group)
        RETURN COUNT(r) AS support

- **Coverage: 76.7**

        // Find valid MEMBER_OF relationships (User -> Group or Domain)
        MATCH (u:User)-[r:MEMBER_OF]->(n)
        WHERE (n:Group OR n:Domain)
        WITH COUNT(r) AS valid_relationships
        
        // Count all MEMBER_OF relationships
        MATCH ()-[r:MEMBER_OF]->()
        WITH valid_relationships, COUNT(r) AS total_relationships
        
        RETURN toFloat(valid_relationships) / total_relationships AS coverage


- **Confidence: 76.6**
  

#### Rule 5. 
**If a relationship has the type `ADMIN_TO`, it must connect a `User` node or a `Group` node to a `Domain` node or a `Computer` node.**

- **Support: 1347**

        MATCH (start)-[r:ADMIN_TO]->(end)
        WHERE (start:User OR start:Group) AND (end:Domain OR end:Computer)
        RETURN COUNT(r) AS valid_admin_to_relationships


- **Coverage: 100**

        // Total number of ADMIN_TO relationships
        MATCH ()-[r:ADMIN_TO]->()
        WITH COUNT(r) AS total_admin_to_relationships
        
        // Number of valid ADMIN_TO relationships
        MATCH (start)-[r:ADMIN_TO]->(end)
        WHERE (start:User OR start:Group) AND (end:Domain OR end:Computer)
        WITH total_admin_to_relationships, COUNT(r) AS valid_admin_to_relationships
        
        // Calculate Coverage
        WITH 
          (valid_admin_to_relationships * 1.0) AS valid_admin_to_relationships, 
          total_admin_to_relationships AS total_admin_to_relationships
        RETURN valid_admin_to_relationships / total_admin_to_relationships * 100 AS coverage

- **Confidence: 100**


#### Rule 6 
**For each `Domain` node, there must be at most one `Domain` node connected via the `PARENT_OF` relationship**

- support = 0
   
#### Rule 7
**For each `Domain` node, there must be at least one `Domain` node connected via the `SUB_DOMAINS` relationship**

- support =  0


#### Rule 8 
**For each `Group` node, there must be at least one `User` node connected via the `MEMBER_OF` relationship**

- **Support: 262**

        // Find Groups with at least one User connected via MEMBER_OF
        MATCH (g:Group)<-[:MEMBER_OF]-(u:User)
        WITH g, COUNT(u) AS user_count
        WHERE user_count >= 1
        RETURN COUNT(g) AS support

- **Coverage: 100**

        // Count Groups with any MEMBER_OF relationships
        MATCH (g:Group)<-[:MEMBER_OF]-(u:User)
        WITH COUNT(DISTINCT g) AS total_groups_with_members
        
        // Count Groups with at least one User connected via MEMBER_OF
        MATCH (g:Group)
        OPTIONAL MATCH (g)<-[:MEMBER_OF]-(u:User)
        WITH total_groups_with_members, g, COUNT(u) AS user_count
        WHERE user_count >= 1
        WITH total_groups_with_members, COUNT(DISTINCT g) AS valid_groups
        
        // Calculate Coverage
        RETURN valid_groups / total_groups_with_members * 100 AS coverage

- **Confidence: 100**


#### Rule 9. 
**For each `Domain` node, there must be at least one `OU` node connected via the `CONTAINS` relationship**

- **Support: 1**

        // Find Domains with at least one OU connected via CONTAINS
        MATCH (d:Domain)-[:CONTAINS]->(ou:OU)
        WITH d, COUNT(ou) AS ou_count
        WHERE ou_count >= 1
        RETURN COUNT(d) AS support


- **Coverage: 100**

        // Count Domains with any CONTAINS relationships
        MATCH (d:Domain)-[:CONTAINS]->(ou:OU)
        WITH COUNT(DISTINCT d) AS total_domains_with_ous
        
        // Count all Domains
        MATCH (d:Domain)
        WITH total_domains_with_ous, COUNT(DISTINCT d) AS total_domains
        
        // Calculate Coverage
        RETURN total_domains_with_ous / total_domains * 100 AS coverage


- **Confidence: 100**

#### Rule 10
**For each `GPO` node, there must be exactly one `OU` node connected via the `AFFECTS` relationship**

- Support =  0

### Few-show Prompt

In [36]:
few_shot_prompt = """
    Examples of consistency Rules:
    
    1. Unique `neo4jImportId` per node: Each node should have a unique `neo4jImportId` property
    2. Unique `objectid` per node: Each node should have a unique `objectid` property.
    3. Only allowed node labels are `User`, `Group`, `Domain`, `OU`, `GPO`, and `Computer`

    Task: Generate new rules to ensure consistency and accuracy in the graph database, considering all node types and relationships. 
    For each consistency rule you identify, provide a clear description of the rule and the corresponding Cypher query for checking
    
    Requirements:
    
    Ensure data consistency across all nodes and relationships.
    Avoid rules that apply to only one type of node or relationship broader context.
"""

### LLAMA

In [37]:
rules_fs_llama = generator(encoded_data, few_shot_prompt, "llama3-70b-8192", 8000)

In [38]:
print(rules_fs_llama)

Based on the provided Neo4j database dump, I'll help you identify consistency rules and provide Cypher queries to check them.

**Rule 1: Unique `neo4jImportId` per node**
Description: Each node should have a unique `neo4jImportId` property.
Cypher query: `MATCH (n) WHERE n.neo4jImportId IS NOT NULL WITH n.neo4jImportId AS id, COUNT(*) AS count WHERE count > 1 RETURN id`

**Rule 2: Unique `objectid` per node**
Description: Each node should have a unique `objectid` property.
Cypher query: `MATCH (n) WHERE n.objectid IS NOT NULL WITH n.objectid AS objectId, COUNT(*) AS count WHERE count > 1 RETURN objectId`

**Rule 3: Only allowed node labels are `User`, `Group`, `Domain`, `OU`, `GPO`, and `Computer`**
Description: Only the specified labels are allowed for nodes.
Cypher query: `MATCH (n) WHERE NOT n:User AND NOT n:Group AND NOT n:Domain AND NOT n:OU AND NOT n:GPO AND NOT n:Computer RETURN n`

**Rule 4: `owned` property consistency**
Description: The `owned` property should be a boolean va

### Evaluation

#### Rule 1: Unique `neo4jImportId` per node
**Description: Each node should have a unique `neo4jImportId` property.**

- **Calculate support (953)**
  
        MATCH (n)
        WITH n.neo4jImportId AS id, COUNT(n) AS countNodes
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN COUNT(id) AS support

- **Calculate coverage (1.0)**
  
        MATCH (n)
        WITH COUNT(n) AS totalNodes
        MATCH (n)
        WITH n.neo4jImportId AS id, COUNT(n) AS countNodes, totalNodes
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN toFloat(COUNT(id)) / totalNodes AS coverage

- **Confidence: 1.0**

        MATCH (n)
        WHERE n.neo4jImportId IS NOT NULL
        WITH COUNT(n) AS totalNodesWithId
        MATCH (n)
        WITH n.neo4jImportId AS id, COUNT(n) AS countNodes, totalNodesWithId
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN toFloat(COUNT(id)) / totalNodesWithId AS confidence


#### Rule 2: Unique `objectid` per node
**Description: Each node should have a unique `objectid` property**

- **Calculate support (948)**
  
        MATCH (n)
        WITH n.objectid AS id, COUNT(n) AS countNodes
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN COUNT(id) AS support

- **Calculate coverage (0.995)**
  
        MATCH (n)
        WITH COUNT(n) AS totalNodes
        MATCH (n)
        WITH n.objectid AS id, COUNT(n) AS countNodes, totalNodes
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN toFloat(COUNT(id)) / totalNodes AS coverage


- **Calculate confidence (0.998)**
  
        MATCH (n)
        WHERE n.objectid IS NOT NULL
        WITH COUNT(n) AS totalNodesWithId
        MATCH (n)
        WITH n.objectid AS id, COUNT(n) AS countNodes, totalNodesWithId
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN toFloat(COUNT(id)) / totalNodesWithId AS confidence




#### Rule 3: Only allowed node labels are `User`, `Group`, `Domain`, `OU`, `GPO`, and `Computer`
**Description: Only the specified labels are allowed for nodes**

- **support - 953**
  
        MATCH (n)
        WHERE  (n:User OR n:Group OR n:Domain OR n:OU OR n:GPO OR n:Computer)
        RETURN COUNT(n) AS support

-  **coverage - 1.0**
  
        MATCH (n)
        WITH COUNT(n) AS totalNodes
        MATCH (n)
        WHERE n:User OR n:Group OR n:Domain OR n:OU OR n:GPO OR n:Computer
        RETURN toFloat(COUNT(n)) / totalNodes AS coverage
        

- **confidence - 1.0**
  
        MATCH (n)
        WHERE NOT (n:User OR n:Group OR n:Domain OR n:OU OR n:GPO OR n:Computer)
        WITH COUNT(n) AS nonConformingNodes
        MATCH (n)
        WHERE n:User OR n:Group OR n:Domain OR n:OU OR n:GPO OR n:Computer
        RETURN toFloat(COUNT(n)) / (COUNT(n) + nonConformingNodes) AS confidence



**Rule 4: `owned` property consistency**
Description: The `owned` property should be a boolean value.

- **Calculate support (603)**
  
        MATCH (n)
        WHERE EXISTS(n.owned) AND (n.owned = true OR n.owned = false)
        RETURN COUNT(n) AS support


- **Calculate coverage (0.63)**
  
        MATCH (n)
        WITH COUNT(n) AS totalNodes
        MATCH (n)
        WHERE EXISTS(n.owned) AND (n.owned = true OR n.owned = false)
        RETURN toFloat(COUNT(n)) / totalNodes AS coverage


	
- **Calculate confidence (1.0)**
  
        MATCH (n)
        WHERE EXISTS(n.owned)
        WITH COUNT(n) AS totalNodesWithOwned
        MATCH (n)
        WHERE EXISTS(n.owned) AND (n.owned = true OR n.owned = false)
        RETURN toFloat(COUNT(n)) / totalNodesWithOwned AS confidence


**Rule 5: `domain` property consistency**
Description: The `domain` property should be a string value and match the format of a domain name (e.g., `TestCompany.Local`).


- **Calculate support (953)**

        MATCH (n)
        WHERE EXISTS(n.domain) AND toString(n.domain) = n.domain
        RETURN COUNT(n) AS support

- **Calculate coverage (1.0)**
  
        MATCH (n)
        WHERE EXISTS(n.domain)
        WITH COUNT(n) AS totalNodesWithDomain
        MATCH (n)
        WHERE EXISTS(n.domain) AND toString(n.domain) = n.domain
        RETURN toFloat(COUNT(n)) / totalNodesWithDomain AS coverage

- **Calculate confidence (1.0)**
  
        MATCH (n)
        WHERE EXISTS(n.domain)
        WITH COUNT(n) AS totalNodesWithDomain
        MATCH (n)
        WHERE EXISTS(n.domain) AND toString(n.domain) = n.domain
        RETURN toFloat(COUNT(n)) / totalNodesWithDomain AS confidence


#### Rule 6: `operatingsystem` property consistency
**Description: The `operatingsystem` property should be a string value and match one of the expected operating system names**

- **Calculate support (300)**
  
        MATCH (n)
        WHERE EXISTS(n.operatingsystem) AND toString(n.operatingsystem) = n.operatingsystem
        RETURN COUNT(n) AS support

- **Calculate coverage (1.0)**
  
        MATCH (n)
        WHERE EXISTS(n.operatingsystem)
        WITH COUNT(n) AS totalNodesWithOperatingSystem
        MATCH (n)
        WHERE EXISTS(n.operatingsystem) AND toString(n.operatingsystem) = n.operatingsystem
        RETURN toFloat(COUNT(n)) / totalNodesWithOperatingSystem AS coverage
        	
- **Calculate confidence (1.0)**
  
        MATCH (n)
        WHERE EXISTS(n.operatingsystem)
        WITH COUNT(n) AS totalNodesWithOperatingSystem
        MATCH (n)
        WHERE EXISTS(n.operatingsystem) AND toString(n.operatingsystem) = n.operatingsystem
        RETURN toFloat(COUNT(n)) / totalNodesWithOperatingSystem AS confidence



#### Rule 7: `name` property consistency
**Description: The `name` property should be a string value and match the format of a username or computer name**

- **Calculate support (953)**
  
        MATCH (n)
        WHERE EXISTS(n.name) AND toString(n.name) = n.name
        RETURN COUNT(n) AS support

- **Calculate coverage (1.0)**
  
        MATCH (n)
        WHERE EXISTS(n.name)
        WITH COUNT(n) AS totalNodesWithName
        MATCH (n)
        WHERE EXISTS(n.name) AND toString(n.name) = n.name
        RETURN toFloat(COUNT(n)) / totalNodesWithName AS coverage
        	
- **Calculate confidence (1.0)**
  
        MATCH (n)
        WHERE EXISTS(n.name)
        WITH COUNT(n) AS totalNodesWithName
        MATCH (n)
        WHERE EXISTS(n.name) AND toString(n.name) = n.name
        RETURN toFloat(COUNT(n)) / totalNodesWithName AS confidence

#### Rule 8: `enabled` property consistency
**Description: The `enabled` property should be a boolean value**

- **Calculate support (600)**
  
        MATCH (n)
        WHERE EXISTS(n.enabled) AND (n.enabled = true OR n.enabled = false)
        RETURN COUNT(n) AS support

- **Calculate coverage (0.63)**
  
        MATCH (n)
        WITH COUNT(n) AS totalNodes
        MATCH (n)
        WHERE EXISTS(n.enabled) AND (n.enabled = true OR n.enabled = false)
        RETURN toFloat(COUNT(n)) / totalNodes AS coverage


	
- **Calculate confidence (1.0)**
  
        MATCH (n)
        WHERE EXISTS(n.enabled)
        WITH COUNT(n) AS totalNodesWithOwned
        MATCH (n)
        WHERE EXISTS(n.enabled) AND (n.enabled = true OR n.enabled = false)
        RETURN toFloat(COUNT(n)) / totalNodesWithOwned AS confidence


#### Rule 9: Relationship types consistency
**Description: Only allowed relationship types are `HAS_SESSION`, `MEMBER_OF`, `CONTAINS`, `GP_LINK`, `ADMIN_TO`, `CAN_RDP`, `EXECUTE_DCOM`, `GENERIC_WRITE`, `GET_CHANGES`, `OWNS`, `WRITE_DACL`, and `WRITE_OWNER`**

- **support - 3762**
  
        MATCH ()-[r]->()
        WHERE  (r:HAS_SESSION OR r:MEMBER_OF OR r:CONTAINS OR r:GP_LINK OR r:ADMIN_TO OR r:CAN_RDP OR r:EXECUTE_DCOM OR r:GENERIC_WRITE OR r:GET_CHANGES OR r:GET_CHANGES OR r:OWNS OR r:WRITE_DACL OR r:WRITE_OWNER)
        RETURN COUNT(r) AS support

-  **coverage - 0.80**
  
        MATCH ()-[r]->()
        WITH COUNT(r) AS totalNodes
        MATCH ()-[r]->()
        WHERE r:HAS_SESSION OR r:MEMBER_OF OR r:CONTAINS OR r:GP_LINK OR r:ADMIN_TO OR r:CAN_RDP OR r:EXECUTE_DCOM OR r:GENERIC_WRITE OR r:GET_CHANGES OR r:GET_CHANGES OR r:OWNS OR r:WRITE_DACL OR r:WRITE_OWNER
        RETURN toFloat(COUNT(r)) / totalNodes AS coverage
        

- **confidence - 0.81**
  
        MATCH ()-[r]->()
        WHERE NOT (r:HAS_SESSION OR r:MEMBER_OF OR r:CONTAINS OR r:GP_LINK OR r:ADMIN_TO OR r:CAN_RDP OR r:EXECUTE_DCOM OR r:GENERIC_WRITE OR r:GET_CHANGES OR r:GET_CHANGES OR r:OWNS OR r:WRITE_DACL OR r:WRITE_OWNER)
        WITH COUNT(r) AS nonConformingNodes
        MATCH ()-[r]->()
        WHERE r:HAS_SESSION OR r:MEMBER_OF OR r:CONTAINS OR r:GP_LINK OR r:ADMIN_TO OR r:CAN_RDP OR r:EXECUTE_DCOM OR r:GENERIC_WRITE OR r:GET_CHANGES OR r:GET_CHANGES OR r:OWNS OR r:WRITE_DACL OR r:WRITE_OWNER
        RETURN toFloat(COUNT(r)) / (COUNT(r) + nonConformingNodes) AS confidence



### MIXTRAL

In [48]:
rules_fs_mixtral = generator(encoded_data, few_shot_prompt, "mixtral-8x7b-32768", 8000)

In [49]:
print(rules_fs_mixtral)

Based on the provided information, here are some consistency rules and corresponding Cypher queries for checking those rules:

1. Nodes should have a unique `neo4jImportId` property:

   Description: Each node in the database should have a unique `neo4jImportId` value.
   Cypher Query:
   ```sql
   MATCH (n)
   WITH n.neo4jImportId AS importId, COUNT(n) AS count
   WHERE count > 1
   RETURN importId, count
   ```

2. Nodes should have a unique `objectid` property:

   Description: Each node in the database should have a unique `objectid` value.
   Cypher Query:
   ```sql
   MATCH (n)
   WHERE n:Computer OR n:Group OR n:User OR n:GPO
   WITH n.objectid AS objectId, COUNT(n) AS count
   WHERE count > 1
   RETURN objectId, count
   ```

3. Nodes should only have allowed labels:

   Description: Only the allowed node labels (`User`, `Group`, `Domain`, `OU`, `GPO`, and `Computer`) should be present in the database.
   Cypher Query:
   ```sql
   MATCH (n)
   WHERE NOT (n:User OR n:Group OR n

### Evaluation

#### Rule 1. Nodes should have a unique `neo4jImportId` property
**Each node in the database should have a unique `neo4jImportId` value**

- **Calculate support (953)**
  
        MATCH (n)
        WITH n.neo4jImportId AS id, COUNT(n) AS countNodes
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN COUNT(id) AS support

- **Calculate coverage (1.0)**
  
        MATCH (n)
        WITH COUNT(n) AS totalNodes
        MATCH (n)
        WITH n.neo4jImportId AS id, COUNT(n) AS countNodes, totalNodes
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN toFloat(COUNT(id)) / totalNodes AS coverage

- **Confidence: 1.0**

        MATCH (n)
        WHERE n.neo4jImportId IS NOT NULL
        WITH COUNT(n) AS totalNodesWithId
        MATCH (n)
        WITH n.neo4jImportId AS id, COUNT(n) AS countNodes, totalNodesWithId
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN toFloat(COUNT(id)) / totalNodesWithId AS confidence

#### Rule 2. Nodes should have a unique `objectid` property
**Each node in the database should have a unique `objectid` value**


- **Calculate support (948)**
  
        MATCH (n)
        WITH n.objectid AS id, COUNT(n) AS countNodes
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN COUNT(id) AS support

- **Calculate coverage (0.995)**
  
        MATCH (n)
        WITH COUNT(n) AS totalNodes
        MATCH (n)
        WITH n.objectid AS id, COUNT(n) AS countNodes, totalNodes
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN toFloat(COUNT(id)) / totalNodes AS coverage


- **Calculate confidence (0.998)**
  
        MATCH (n)
        WHERE n.objectid IS NOT NULL
        WITH COUNT(n) AS totalNodesWithId
        MATCH (n)
        WITH n.objectid AS id, COUNT(n) AS countNodes, totalNodesWithId
        WHERE id IS NOT NULL AND countNodes = 1
        RETURN toFloat(COUNT(id)) / totalNodesWithId AS confidence





#### Rule 3. Nodes should only have allowed labels

**Description: Only the allowed node labels (`User`, `Group`, `Domain`, `OU`, `GPO`, and `Computer`) should be present in the database.**


  - **support - 953**
  
        MATCH (n)
        WHERE  (n:User OR n:Group OR n:Domain OR n:OU OR n:GPO OR n:Computer)
        RETURN COUNT(n) AS support

-  **coverage - 1.0**
  
        MATCH (n)
        WITH COUNT(n) AS totalNodes
        MATCH (n)
        WHERE n:User OR n:Group OR n:Domain OR n:OU OR n:GPO OR n:Computer
        RETURN toFloat(COUNT(n)) / totalNodes AS coverage
        

- **confidence - 1.0**
  
        MATCH (n)
        WHERE NOT (n:User OR n:Group OR n:Domain OR n:OU OR n:GPO OR n:Computer)
        WITH COUNT(n) AS nonConformingNodes
        MATCH (n)
        WHERE n:User OR n:Group OR n:Domain OR n:OU OR n:GPO OR n:Computer
        RETURN toFloat(COUNT(n)) / (COUNT(n) + nonConformingNodes) AS confidence



#### Rule 4. A `Computer` node cannot be a member of a `Domain` node
**Description: A `Computer` node should not have a relationship with a `Domain` node using the `MEMBER_OF` relationship**

- **Support: 0**

        MATCH (c:Computer)-[r:MEMBER_OF]->(d:Domain)
        RETURN COUNT(*) AS support

- **Coverage: 100**
- **Confidence: 100**


#### Rule 5. GPOs must be linked to either a `Domain` node or an `OU` node:
**Description: A `GPO` node should be linked to either a `Domain` node or an `OU` node through a relationship of `GP_LINKS`**

- **Support: 21**

        // Count the number of GPO nodes linked to Domain or OU nodes
        MATCH (g:GPO)-[:GP_LINK]->(target)
        WHERE (target:Domain OR target:OU)
        RETURN COUNT(DISTINCT g) AS support

- **coverage: 60%**

- **Confidence: 60%**


